# Projeto Fase 6 – Rede Neural YOLO

Este notebook implementa um sistema de visão computacional utilizando **YOLOv5** para detecção de objetos. Como parte da fase 6 do curso, montamos um dataset sintético com duas classes (círculo vermelho e quadrado azul) e seguimos as etapas de preparação, treinamento, validação e teste.

**Estrutura do notebook:**

1. Preparação do dataset (geração de imagens e rótulos)
2. Configuração do arquivo `dataset.yaml`
3. (Opcional) Instalação e configuração do YOLOv5 no Google Colab
4. Treinamento do modelo com diferentes números de épocas
5. Validação e teste
6. Conclusões



In [ ]:
import os, random
from PIL import Image, ImageDraw

# Cria a estrutura de pastas
def generate_dataset(base_path, num_train=32, num_val=4, num_test=4):
    os.makedirs(os.path.join(base_path, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images', 'test'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'test'), exist_ok=True)

    def create_image(shape):
        img = Image.new('RGB', (256,256), 'white')
        draw = ImageDraw.Draw(img)
        x0 = random.randint(30,150)
        y0 = random.randint(30,150)
        size = random.randint(50,120)
        x1 = x0 + size
        y1 = y0 + size
        if shape == 'circle':
            draw.ellipse([x0, y0, x1, y1], fill='red', outline='black')
            class_id = 0
        else:
            draw.rectangle([x0, y0, x1, y1], fill='blue', outline='black')
            class_id = 1
        x_center = ((x0 + x1)/2) / 256
        y_center = ((y0 + y1)/2) / 256
        width = (x1 - x0) / 256
        height = (y1 - y0) / 256
        return img, f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}
"

    for subset, n in [('train',num_train), ('val', num_val), ('test', num_test)]:
        for i in range(n):
            # círculo vermelho
            img, label = create_image('circle')
            filename = f"circle_{subset}_{i}.png"
            img.save(os.path.join(base_path, 'images', subset, filename))
            with open(os.path.join(base_path, 'labels', subset, filename.replace('.png','.txt')), 'w') as f:
                f.write(label)
            # quadrado azul
            img, label = create_image('square')
            filename = f"square_{subset}_{i}.png"
            img.save(os.path.join(base_path, 'images', subset, filename))
            with open(os.path.join(base_path, 'labels', subset, filename.replace('.png','.txt')), 'w') as f:
                f.write(label)

# Define o caminho base (Drive ou local)
base_dir = '/content/fase6_dataset'

# Gera o dataset
generate_dataset(base_dir)

# Mostra quantas imagens foram geradas
print('Train images:', len(os.listdir(os.path.join(base_dir, 'images', 'train'))))
print('Val images:', len(os.listdir(os.path.join(base_dir, 'images', 'val'))))
print('Test images:', len(os.listdir(os.path.join(base_dir, 'images', 'test'))))


## Instalação do YOLOv5

Para treinar a YOLOv5 no Google Colab, recomendamos clonar o repositório oficial e instalar os requisitos. No Colab, execute:

```python
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
```

Em seguida, copie o arquivo de configuração `dataset.yaml` para a pasta raiz do Drive ou para a pasta do repositório e edite o caminho conforme necessário.


In [ ]:
# Treinamento com 30 épocas
!python yolov5/train.py --data dataset.yaml --weights yolov5s.pt --img 256 --epochs 30 --name shape_yolo_30

# Treinamento com 60 épocas
!python yolov5/train.py --data dataset.yaml --weights yolov5s.pt --img 256 --epochs 60 --name shape_yolo_60


## Avaliação e Conclusões

Após o treinamento, os resultados de validação e teste são salvos em `yolov5/runs/train/shape_yolo_30` e `shape_yolo_60`. Compare as métricas de **precisão (P)**, **recall (R)** e **mAP** para as diferentes quantidades de épocas. Observa-se que aumentar o número de épocas tende a melhorar a precisão até certo ponto, mas também aumenta o tempo de treinamento.

Com base nas métricas obtidas, descreva aqui os pontos fortes e limitações do modelo treinado com 30 e 60 épocas, mencionando a variação de desempenho entre as classes círculo e quadrado.
